# Analítica Web - Informes
#### Alba Márquez Rodríguez

A continuación se presenta la actividad *Informes*

## DATASET: LogsDataiku.csv

Este dataset almacena información de acceso al sitio web de Dataiku, durante 2 meses. Contiene información sobre cada vista de página individual en el sitio web www.Dataiku.com

## Estructura del dataset
El dataset, está en un fichero csv con 18 campos:
- server_ts: fecha de conexión al servidor
- client_ts: fecha de conexión del usuario
- client_addr: dirección IP del usuario
- visitor_id: id asignado al usuario (~ _utma)
- session_id: id de la sesión/visita (~ _utmb)
- location: url de la página a la que se accede
- referer: url de la página desde la que se accedió a location. Si está vacío se considera una búsqueda directa
- user_agent: navegador desde el que se conectó el usuario
- type: tipo de acceso
- visitor_param
- sesion_param:
- event_param:
- br_width: ancho del navegador del usuario
- br_height: alto del navegador del usuario
- sc_width: ancho de la pantalla del usuario (resolución)
- sc_height: alto de la pantalla del usuario (resolución)
- br_lan: idioma del navegador del usuario
- tz_off: time zona. El número indica la diferencia en minutos con la hora GMT . -60 indica zona horaria GMT+1 y 60 indica zona horaria GMT-1

## Ejercicio
A partir de los datos del dataset anterior, generar informes para, en base a dichos datos, poder tomar las siguientes decisiones:
- Qué idiomas debería soportar el sitio web
- Se quiere sacar una promoción durante un tiempo muy limitado. Qué hora sería la más adecuada para llegar a un mayor número de usuarios
- Podemos decidir promocionarnos en los enlaces que vienen de serie con los navegadores. En qué navegador sería más interesante hacernos publicidad


#### Librerías

In [1]:
import pandas as pd
import numpy as np

#### Carga de los datos

Para comenzar cargaremos los datos desde el csv

In [4]:
# Cargar dataset
df = pd.read_csv('LogsDataiku/LogsDataiku.csv')

### Qué idiomas debería soportar el sitio web

Analizando la columna br_lan, que indica el idioma del navegador del usuario, se puede determinar qué idiomas son más comunes entre los usuarios. Se calcula la distribución de los idiomas y nos centramos en los más frecuentes para decidir qué idiomas debería soportar el sitio web.

In [8]:
# Idiomas que debería soportar el sitio web
top_languages = df['br_lang'].value_counts().head(5)
print("Idiomas más comunes entre los usuarios:")
print(top_languages)

Idiomas más comunes entre los usuarios:
br_lang
fr       5033
en-US    2927
fr-fr     748
en-us     668
en-GB     360
Name: count, dtype: int64


Los idiomas más frecuentes son francés e inglés, sin embargo se pueden ver qué otros idiomas también son utilizados en los navegadores de las personas usuarias y, en el caso de no ser demasiados y tener la capacidad de implementarlo, realizarlo.

In [11]:
# Idiomas que debería soportar el sitio web
top_languages = df['br_lang'].value_counts()
print("Idiomas utilizados entre los usuarios:")
print(top_languages)

Idiomas utilizados entre los usuarios:
br_lang
fr       5033
en-US    2927
fr-fr     748
en-us     668
en-GB     360
         ... 
fr-ch       1
sv-se       1
fi-FI       1
ko-KR       1
es-AR       1
Name: count, Length: 68, dtype: int64


Hay muchos idiomas en el que el número de usuarios apenas es representativo, habiendo un total de 68 idiomas y teniendo en cuenta que muchos son el idioma + el dialecto, vamos a agruparlos por la primera parte del idioma

In [13]:
# Idiomas que debería soportar el sitio web
top_languages = df['br_lang'].value_counts()
# Agrupar idiomas por la primera parte antes del -
top_languages = top_languages.groupby(lambda x: x.split('-')[0]).sum()
# Ordenar de mayor a menor
top_languages = top_languages.sort_values(ascending=False)
print("Idiomas utilizados entre los usuarios:")
print(top_languages)

Idiomas utilizados entre los usuarios:
br_lang
fr       6049
en       4151
pt        104
de         90
zh         89
es         87
ru         80
ja         56
pl         24
nl         21
ca         15
tr         14
da         12
cs          9
it          8
bg          7
he          6
th          5
id          4
nb          3
sk          2
C           2
ko          2
en_US       2
hu          2
hr          1
fi          1
fa          1
sv          1
el          1
Name: count, dtype: int64


Ahora tenemos los idiomas y podemos ver que, teniendo en cuenta la agrupación de regiones y dialectos, los más comúnes son:
- Francés
- Inglés
- Portugués
- Alemán
- Chino
- Español
- Ruso
- Japonés
- Polaco
- Holandés

Los idiomas realmente relevantes son el inglés y el francés, del resto no hay muchos usuarios pero se podría estudiar si es un público target para implementar este idioma e intentar aumentar así el número de usuarios de esos países / que hablen esos idiomas.

### Se quiere sacar una promoción durante un tiempo muy limitado. Qué hora sería la más adecuada para llegar a un mayor número de usuarios

Podemos analizar la columna client_ts (fecha de conexión del usuario) para determinar en qué horas del día se registra un mayor número de visitas al sitio web. Calcularemos la cantidad de visitas en cada hora del día y seleccionaremos la hora con el pico más alto de visitas como el momento más adecuado para lanzar la promoción.

In [9]:
# Hora más adecuada para lanzar una promoción
df['client_ts'] = pd.to_datetime(df['client_ts'])
hourly_visits = df['client_ts'].dt.hour.value_counts()
best_hour = hourly_visits.idxmax()
print("\nHora más adecuada para lanzar una promoción:", best_hour, "hrs")


Hora más adecuada para lanzar una promoción: 14 hrs


### Podemos decidir promocionarnos en los enlaces que vienen de serie con los navegadores. En qué navegador sería más interesante hacernos publicidad

Analizando la columna referer se pueden identificar los navegadores desde los cuales se originan los enlaces de referencia. Luego, se puede calcular la proporción de visitas que provienen de cada navegador y podremos quedarnos con el navegador con mayor proporción de visitas.

In [14]:
# Decidir promocionarse en los enlaces que vienen de serie con los navegadores
referer_browser = df['referer'].str.extract(r'://(.*?)/', expand=False)
browser_traffic = referer_browser.value_counts(normalize=True)
# Print 5 most common browsers
print("\nTráfico por navegador:")
print(browser_traffic.head(5))


Tráfico por navegador:
referer
www.dataiku.com         0.565489
dataiku.com             0.171830
www.google.fr           0.075468
www.google.com          0.046466
www.journaldunet.com    0.026507
Name: proportion, dtype: float64


En este caso el mayor tráfico viene desde la misma página web, lo que podría decir que la mayoría del tráfico viene de los usuarios que usan la página. Queriendo decir que interactúan de forma activa con la página y su contenido. Por otro lado, el navegador externo más utilizado es el de google, de procedencia francesa o general (.fr y .com)